### The Challenge

Use python code to create a text classification model that uses a tf-idf feature set to
classify documents in the Twenty News Groups Data Set (https://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups).

- Do not use any pre-built packages to generate the tf-idf features (i.e. you can use numpy, pandas etc. but not scikit)
- Choose 3 different classification algorithms and compare how each of them does on the task
- Of the 3 you test out, choose the one that performs the best for your final code
- Provide a brief (less than 300 words) write-up explaining the following points:
  - How well does your model perform at the given task?
  - How did you compare the different classification algorithms?
  - What are some of the limitations of the model?

Publish your code to a private GitHub repo and send us an invitation!

### Project Dependencies

In [1]:
import os
import numpy as np
import re, string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn import metrics

# We need a progress bar because this term-frequency 
# stuff is probably going to take a while.
from tqdm import tqdm

### Data Ingestion

Bring docs into memory and strip out the metadata, which happens to be everything up to the first double newline characters.

In [2]:
groups = [group for group in os.listdir('../data/20_newsgroups') if group[0] != '.']
len(groups)

20

There's 20 groups, but we'll just pick a few to work with mainly due to memory and time constraints:

- alt.atheism
- comp.graphics
- misc.forsale
- sci.crypt

In [3]:
groups = ['alt.atheism', 'comp.graphics', 'misc.forsale', 'rec.autos', 'sci.crypt']

# create a dictionary of all documents, preserving the name of the group 
# in the key
docs_per_group = dict()
all_docs = dict()

for group in groups:
    docs = os.listdir('../data/20_newsgroups/' + group)
    docs_per_group[group] = len(docs)
    for doc in docs:
        doc_content = ''
        for line in open(os.path.join('../data/20_newsgroups/', group, doc), encoding = "ISO-8859-1"):
            doc_content += line

        # for every doc, strip out the metadata, which is up to 
        # the first 2 newline characters in a row
        doc_content = ''.join(doc_content.split('\n\n')[1:])

        if doc_content != '':
            doc_name = group + '_' + doc
            all_docs[doc_name] = doc_content

In [4]:
len(all_docs)

4998

Check the balance of the documents.

In [5]:
docs_per_group

{'alt.atheism': 1000,
 'comp.graphics': 1000,
 'misc.forsale': 1000,
 'rec.autos': 1000,
 'sci.crypt': 1000}

Freeze the order of documents.

In [6]:
all_doc_names = list(all_docs.keys())

### Split into Train/Test Sets

Create the random `80:20` `train:test` splits and vectorize the target variables.

**Note:** It's reasonable for the model to not be aware of the vocabulary that's in the test set, so when extracting the features from the test set, only use the vocabulary set built up by the training data. There will be tokens that will not be used for creating features during testing.

In [7]:
NUM_DOCS = len(all_doc_names)

TRAIN_SPLIT = 0.8
TEST_SPLIT = 0.2

# split documents into 80/20
train_doc_indices = np.random.choice(NUM_DOCS, np.floor(NUM_DOCS * TRAIN_SPLIT).astype(int), replace=False)
test_doc_indices = np.delete(np.arange(NUM_DOCS), train_doc_indices) # take the remainder

# vectorize targets
string_targets = np.asarray([doc.split('_')[0] for doc in all_doc_names])
uniques, vectorized_targets = np.unique(string_targets, return_inverse=True)

vectorized_targets

array([0, 0, 0, ..., 4, 4, 4])

In [8]:
train_targets = []
for i in train_doc_indices:
    train_targets.append(vectorized_targets[i])

train_targets = np.asarray(train_targets)
    
test_targets = []
for i in test_doc_indices:
    test_targets.append(vectorized_targets[i])
    
test_targets = np.asarray(test_targets)

Target names:

In [9]:
uniques

array(['alt.atheism', 'comp.graphics', 'misc.forsale', 'rec.autos',
       'sci.crypt'],
      dtype='<U13')

### Generate TF-IDF Features

According to [Wikipedia](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), The simplest form of term frequency is calculated using the "raw count of a term in a document", or $tf(t, d)=f_{t,d}$. If the term frequency is adjusted for document length, $tf(t, d)=f_{t,d} /$ `number of words in d`.

The inverse document frequency is calculated by using $idf(t, D)=\log(\frac{N}{1 + |\{d \in D : t \in d\}|})$, where $N$ is the total number of documents in the corpus, and $|\{d \in D : t \in d\}|$ is the number of documents where the term t appears.

In [10]:
# in addition to standard stopwords by nltk, want to remove punctuation
stop_words = stopwords.words('english') + list(string.punctuation)

Create a function for preprocessing each document, which includes:
- tokenization
- lowercasing
- stopword removal

In [11]:
def preprocess_doc(doc, stop_words):
    """
    Remove all tokens with digits and stopwords.
    """
    preprocessed_tokens = []
    tokens = word_tokenize(doc)
    for token in tokens:
        token = re.sub('[\W_]', '', token.lower())
        if token not in stop_words and token != '' and not token.isdigit():
            preprocessed_tokens.append(token)
    return preprocessed_tokens

Set up three functions below, which all rely on each other for calculating the overall tf-idf feature for each document. Each document's tf-idf features will be mapped to a target, which is the type of document (one of the groups that we chose: `alt.atheism`, `comp.graphics`, `misc.forsale`, or `sci.crypt`).

In [12]:
def calculate_tf(token, doc):
    """
    (str, list) --> float
    
    Calculate the token's term frequency.
    
    This function assumes that the contents of the doc have been tokenized
    """
    return float(doc.count(token) / (len(doc) + 1))

def calculate_idf(array, num_docs):
    """
    (np.array, int) --> np.array
    
    Calculate the inverse document frequency, which is:
    the log of the number of documents in the corpus divided
    by the number of docs where the term appears
    
    the term_index in the matrix is given, the sum of which 
    gives the number of docs in which the term appears
    """
    
    return np.log(num_docs / (array + 1).astype(float))

def calculate_tf_idf(token, doc, idf_array, vocabulary_index):
    """
    (str, list, np.array, dict)
    
    Calculates the term frequency-inverse document frequency.
    
    """
    if token not in vocabulary_index:
        return float(0)
    else:
        return calculate_tf(token, doc) * idf_array[vocabulary_index[token]]

In [13]:
# build the feature sets for train/test, 
# removing all non-alphanumeric characters, digits and stopwords

all_train_tokens = set()
all_test_tokens = set()

for i in train_doc_indices:
    tokens = preprocess_doc(all_docs[all_doc_names[i]], stop_words)
    all_train_tokens.update(tokens)

for j in test_doc_indices:
    tokens = preprocess_doc(all_docs[all_doc_names[j]], stop_words)
    all_test_tokens.update(tokens)
    
all_train_tokens = list(all_train_tokens)
all_test_tokens = list(all_test_tokens)

In [14]:
len(all_train_tokens)

51881

In [15]:
len(all_test_tokens)

23333

In [16]:
# create vocabulary lookup dictionaries
# only for training, because test vocabulary is not known!

train_vocabulary_index = dict()

i = 0
for token in all_train_tokens:
    train_vocabulary_index[token] = i
    i += 1

In [17]:
len(train_vocabulary_index)

51881

Build the inverse document frequency arrays for train and test.

In [18]:
train_idf_array = np.zeros(len(all_train_tokens))

# train
for i in tqdm(train_doc_indices):
    preprocessed_tokens = set(preprocess_doc(all_docs[all_doc_names[i]], stop_words))
    for token in preprocessed_tokens:
        train_idf_array[train_vocabulary_index[token]] += 1.0

100%|██████████| 3998/3998 [00:13<00:00, 307.43it/s]


In [19]:
test_idf_array = np.zeros(len(all_train_tokens))

keyerrors = 0
# test
for i in tqdm(test_doc_indices):
    preprocessed_tokens = set(preprocess_doc(all_docs[all_doc_names[i]], stop_words))
    for token in preprocessed_tokens:
        try:
            test_idf_array[train_vocabulary_index[token]] += 1.0
        except KeyError:
            keyerrors += 1

100%|██████████| 1000/1000 [00:03<00:00, 286.62it/s]


The number of tokens that were in the test set but not in the training set are:

In [20]:
keyerrors

6577

In [21]:
train_idf_array = calculate_idf(train_idf_array, len(train_doc_indices))
test_idf_array = calculate_idf(test_idf_array, len(test_doc_indices))

For every document and for every token, calculate the TF-IDF.

In [22]:
train_tf_idf_matrix = np.zeros(shape=(len(train_doc_indices), len(all_train_tokens)))

# train
i = 0
for index in tqdm(train_doc_indices):
    tokenized_doc = list(preprocess_doc(all_docs[all_doc_names[index]], stop_words))
    for j in range(len(all_train_tokens)):
        train_tf_idf_matrix[i][j] = calculate_tf_idf(all_train_tokens[j], tokenized_doc, train_idf_array, train_vocabulary_index)
    i += 1

100%|██████████| 3998/3998 [11:49<00:00,  6.33it/s]


In [23]:
test_tf_idf_matrix = np.zeros(shape=(len(test_doc_indices), len(all_train_tokens)))

test_tf_idf_keyerrors = 0

# test
i = 0
for index in tqdm(test_doc_indices):
    tokenized_doc = list(preprocess_doc(all_docs[all_doc_names[index]], stop_words))
    for j in range(len(all_test_tokens)):
        try:
            test_tf_idf_matrix[i][train_vocabulary_index[all_test_tokens[j]]] = calculate_tf_idf(all_test_tokens[j], tokenized_doc, test_idf_array, train_vocabulary_index)
        except KeyError:
            test_tf_idf_keyerrors += 1
    i += 1

100%|██████████| 1000/1000 [01:19<00:00, 12.63it/s]


In [24]:
test_tf_idf_keyerrors

6206000

Check that the shapes of the matrices match up with the train and test tf-idf matrices.

In [25]:
train_tf_idf_matrix.shape

(3998, 51881)

In [26]:
test_tf_idf_matrix.shape

(1000, 51881)

In [27]:
train_targets.shape

(3998,)

In [28]:
test_targets.shape

(1000,)

### Classification and Model Evaluation

I chose 3 models to test the classification of text:
- Naive Bayes
- SVM
- KNN

#### Naive Bayes

One of the simplest of them all, the Naive Bayes classifier is "naive" because of the assumption that all of the features in a dataset are mutually independent.

In [29]:
from sklearn.naive_bayes import MultinomialNB

nb_clf = MultinomialNB().fit(train_tf_idf_matrix, train_targets)
nb_predicted = nb_clf.predict(test_tf_idf_matrix)
np.mean(nb_predicted == test_targets)

0.95699999999999996

- **Precision**: % of selected items that are correct
- **Recall**: % of correct items that are selected
- **F1-score**: the harmonic mean of precision and recall

In [30]:
print(metrics.classification_report(test_targets, nb_predicted, target_names=uniques))

               precision    recall  f1-score   support

  alt.atheism       0.98      0.97      0.97       213
comp.graphics       0.94      0.95      0.95       211
 misc.forsale       0.96      0.93      0.94       197
    rec.autos       0.94      0.97      0.95       182
    sci.crypt       0.97      0.96      0.97       197

  avg / total       0.96      0.96      0.96      1000



In [31]:
metrics.confusion_matrix(test_targets, nb_predicted)

array([[206,   2,   0,   2,   3],
       [  2, 201,   6,   1,   1],
       [  0,   4, 183,   9,   1],
       [  0,   3,   1, 177,   1],
       [  2,   4,   1,   0, 190]])

#### Linear SVM

In [32]:
from sklearn.linear_model import SGDClassifier

svm_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
svm_clf.fit(train_tf_idf_matrix, train_targets)
svm_predicted = svm_clf.predict(test_tf_idf_matrix)
np.mean(svm_predicted == test_targets)

0.94999999999999996

In [33]:
print(metrics.classification_report(test_targets, svm_predicted, target_names=uniques))

               precision    recall  f1-score   support

  alt.atheism       0.99      0.96      0.97       213
comp.graphics       0.88      0.96      0.92       211
 misc.forsale       0.93      0.94      0.94       197
    rec.autos       0.98      0.94      0.96       182
    sci.crypt       0.98      0.94      0.96       197

  avg / total       0.95      0.95      0.95      1000



In [34]:
metrics.confusion_matrix(test_targets, svm_predicted)

array([[205,   6,   0,   1,   1],
       [  2, 202,   6,   1,   0],
       [  0,   9, 186,   1,   1],
       [  0,   5,   5, 171,   1],
       [  1,   8,   2,   0, 186]])

#### KNN

In [35]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(train_tf_idf_matrix, train_targets)
knn_predicted = knn_clf.predict(test_tf_idf_matrix)
np.mean(knn_predicted == test_targets)

0.34100000000000003

In [36]:
print(metrics.classification_report(test_targets, knn_predicted, target_names=uniques))

               precision    recall  f1-score   support

  alt.atheism       0.95      0.24      0.39       213
comp.graphics       0.82      0.26      0.39       211
 misc.forsale       0.23      0.99      0.38       197
    rec.autos       1.00      0.13      0.22       182
    sci.crypt       1.00      0.08      0.15       197

  avg / total       0.80      0.34      0.31      1000



In [37]:
metrics.confusion_matrix(test_targets, knn_predicted)

array([[ 52,   3, 158,   0,   0],
       [  2,  54, 155,   0,   0],
       [  0,   1, 196,   0,   0],
       [  0,   4, 155,  23,   0],
       [  1,   4, 176,   0,  16]])

The KNN is very slow and not accurate at all for its current parameter setting of 5 neighbors. I don't think it's worth the time trying to do any sort of grid search because the naive bayes and SVM perform so well out of the box.

### Writeup

The model that performed the best was the multinomial Naive Bayes classifier, receiving an accuracy score of 95.7%. I think that the large part of the performance boost came from the initial feature extraction, which included preprocessing steps such as stopword and punctuation removal, as well as the fact that I had started off with a balanced dataset. 

The different classification algorithms were compared using their overall accuracy, precision, recall, F1 scores and through a visual perusal of the confusion matrix. The multinomial Naive Bayes classifier's precision, recall and F1-scores were all 96%. 

The main limitation of the Naive Bayes classifier is its primary assumption that all of the data that it models are independent and identically distributed random variables, which would mean that it is unable to model data that violates this assumption.

# Future To-Do

- proper cross validation (k-fold)
- prettier confusion matrices with `pandas_ml`